# Feature engineering

Colunas FUNIL_ASSINATURA_PIPEDRIVE_status até ATENDIMENTOS_AGENDA_Datas Atendimento Médico

In [40]:
import pandas as pd

data = pd.read_csv("../data/data-preprocessed.csv")

## Coluna extra: Tempo de permanência

Coluna "stay_time" criada para calcular tempo total do usuário na plataforma. Utiliza lost_time - start_of_service ou lost_time - contract_start_date, se os dados de start_of_service forem nulos. 

In [ ]:
import pandas as pd
from datetime import datetime

tempo_permanencia = []

for indice, valor in data["FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"].items():
  if pd.notna(valor):
      index = data.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"].find(";")
      if index != -1:
        data.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"] = data.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"][:index]

for indice, valor in data["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
  if pd.notna(valor):
    if data.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] != "Em aberto":	
      if pd.notna(data.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"]):
        tempo_1 = datetime.strptime(data.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"], '%Y-%m-%d')
        tempo_2 = datetime.strptime(data.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"], '%Y-%m-%d')
        tempo_permanencia.append(str(tempo_1 - tempo_2))
      else:
        tempo_1 = datetime.strptime(data.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"], '%Y-%m-%d')
        tempo_2 = datetime.strptime(data.loc[indice, "PESSOA_PIPEDRIVE_contract_start_date"], '%Y-%m-%d')
        tempo_permanencia.append(str(tempo_1 - tempo_2))
    else:
      tempo_permanencia.append("Em aberto")

data['stay_time'] = tempo_permanencia

for indice, valor in data["stay_time"].items():
  if valor != "Em aberto":
    index = data.loc[indice, "stay_time"].find(",")
    if index != -1:
      data.loc[indice, "stay_time"] = data.loc[indice, "stay_time"][:index]

data['stay_time']



## Coluna extra: Último processo de onboarding concluído

Coluna criada usando as informações de stay_in_pipeline_stages_welcome, stay_in_pipeline_stages_first_meeting e stay_in_pipeline_stages_whoqol. Mostra o último processo concluído. 


In [ ]:
ultimos_estados = []

for indice, valor in data["FUNIL_ONBOARDING_PIPEDRIVE_add_time"].items():
    if valor != "Não iniciado":
        if pd.notna(data.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_welcome"]):
            if pd.notna(data.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_first_meeting"]):
                if pd.notna(data.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_whoqol"]):
                    ultimos_estados.append("Questionário")
                else:
                    ultimos_estados.append("Primeira reunião")
            else:
                ultimos_estados.append("Boas-vindas")
        
    else:
        ultimos_estados.append("Não iniciado")

data["last_stage_concluded"] = ultimos_estados

data.drop(["FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_welcome", "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_first_meeting", "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_whoqol"], axis=1, inplace=True)

## Coluna extra: duração de onboarding

Combina as informações de won_time e lost_time em uma única coluna. Se tiver lost time, usa lost time; se tiver won time, usa won time; se add time for null, usar "não iniciado"; do contrario, usar "em aberto".


In [ ]:
## Coluna extra: duração de onboarding

Combina as informações de won_time e lost_time em uma única coluna. Se tiver lost time, usa lost time; se tiver won time, usa won time; se add time for null, usar "não iniciado"; do contrario, usar "em aberto".

tempo = []

for indice, valor in data["FUNIL_ONBOARDING_PIPEDRIVE_add_time"].items():
    if valor != "Não iniciado":
        if pd.notna(data.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_time"]) == True:
            tempo.append(data.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_time"])
        elif pd.notna(data.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_won_time"]) == True:
            tempo.append(data.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_won_time"])
        else:
            tempo.append("Em aberto")
    else:
        tempo.append("Não iniciado")

data['process_time'] = tempo



for indice, valor in data["process_time"].items():
  if data.loc[indice, "process_time"] != "Não iniciado" and data.loc[indice, "process_time"] != "Em aberto":
    data.loc[indice, "process_time"] = pd.to_datetime(data.loc[indice, "process_time"], format='%Y-%m-%d', errors='coerce')
    data.loc[indice, "process_time"] = data.loc[indice, "process_time"].strftime('%Y-%m-%d')

data.drop(["FUNIL_ONBOARDING_PIPEDRIVE_won_time", "FUNIL_ONBOARDING_PIPEDRIVE_lost_time"], axis=1, inplace=True)



## FUNIL_ASSINATURA_PIPEDRIVE_status

Transforma dados da coluna em one not encoding

In [41]:
data['FUNIL_ASSINATURA_PIPEDRIVE_status'].value_counts()

won     544
lost    428
Name: FUNIL_ASSINATURA_PIPEDRIVE_status, dtype: int64

In [42]:
one_hot_encoded = pd.get_dummies(data['FUNIL_ASSINATURA_PIPEDRIVE_status'], prefix='status')
data = pd.concat([data, one_hot_encoded], axis=1)

## FUNIL_ONBOARDING_PIPEDRIVE_status

One-hot encoding aplicado nessa coluna

In [ ]:
data['FUNIL_ONBOARDING_PIPEDRIVE_status'].value_counts()

won             579
lost            258
open             99
Não iniciado     36
Name: FUNIL_ONBOARDING_PIPEDRIVE_status, dtype: int64

In [ ]:
data_status_encoded = pd.get_dummies(data['FUNIL_ONBOARDING_PIPEDRIVE_status'], prefix='Status')
data = pd.concat([data, data_status_encoded], axis=1)

## FUNIL_ONBOARDING_PIPEDRIVE_lost_reason

One hot encoding aplicado na coluna

In [ ]:
data['FUNIL_ONBOARDING_PIPEDRIVE_lost_reason'].value_counts()

Outro                                                                747
[Onboarding] Não retornou aos contatos de resgate                    195
[Onboarding] Não tem interesse em seguir nas etapas do onboarding     18
[Associade] Cancelou assinatura                                       12
Name: FUNIL_ONBOARDING_PIPEDRIVE_lost_reason, dtype: int64

In [ ]:
lost_reason_dummies = pd.get_dummies(data['FUNIL_ONBOARDING_PIPEDRIVE_lost_reason'], prefix='lost_reason')
data = pd.concat([data, lost_reason_dummies], axis=1)


## FUNIL_ASSINATURA_PIPEDRIVE_lost_reason

Faz one hot encoding para motivos de cancelamento de assinatura

In [ ]:
data['FUNIL_ASSINATURA_PIPEDRIVE_lost_reason'].value_counts()

Outro                                                         601
[Assinatura] Desligamento                                     211
[Assinatura] Empresa cancelou o benefício da Ana               53
[Assinatura] Não quer seguir com a Ana                         49
[Assinatura] Precisou cortar custos                            24
[Assinatura] Cancelamento por inadimplência                    18
[Assinatura] Está sem tempo para conciliar os atendimentos     16
Name: FUNIL_ASSINATURA_PIPEDRIVE_lost_reason, dtype: int64

In [ ]:
lost_reason_dummies = pd.get_dummies(data['FUNIL_ASSINATURA_PIPEDRIVE_lost_reason'], prefix='lost_reason')
data = pd.concat([data, lost_reason_dummies], axis=1)

data

,PESSOA_PIPEDRIVE_id_person,PESSOA_PIPEDRIVE_id_gender,PESSOA_PIPEDRIVE_id_marrital_status,PESSOA_PIPEDRIVE_state,PESSOA_PIPEDRIVE_city,PESSOA_PIPEDRIVE_postal_code,PESSOA_PIPEDRIVE_contract_start_date,PESSOA_PIPEDRIVE_contract_end_date,PESSOA_PIPEDRIVE_Canal de Preferência,PESSOA_PIPEDRIVE_notes_count,...,process_time,TWILIO_Data Última Mensagens Outbound Recente,TWILIO_Data Última Ligações Outbound Recente,lost_reason_Outro,lost_reason_[Assinatura] Cancelamento por inadimplência,lost_reason_[Assinatura] Desligamento,lost_reason_[Assinatura] Empresa cancelou o benefício da Ana,lost_reason_[Assinatura] Está sem tempo para conciliar os atendimentos,lost_reason_[Assinatura] Não quer seguir com a Ana,lost_reason_[Assinatura] Precisou cortar custos
0,483,64,80,Minas Gerais,Juiz de Fora,36021-600,2021-01-27,2023-08-26,0,3,...,2022-04-19 15:19:40,1,1,0,0,0,0,0,1,0
1,860,63,80,Paraná,Londrina,86020-410,2022-06-21,2023-09-20,239,3,...,2022-06-27 15:03:30,1,0,1,0,0,0,0,0,0
2,961,63,80,Santa Catarina,Florianópolis,88080-080,2021-07-05,2021-10-01,0,1,...,2021-10-04 12:25:41,0,0,0,0,0,1,0,0,0
3,972,63,82,Minas Gerais,Uberlândia,38400-438,2021-06-28,2023-09-16,0,2,...,2022-05-05 13:14:01,1,0,1,0,0,0,0,0,0
4,973,64,80,Minas Gerais,Uberlândia,38405-364,2021-06-28,2022-02-07,0,1,...,2022-02-07 03:00:00,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,5944,64,80,Paraíba,João Pessoa,58052-310,2023-11-06,NaN,239,0,...,Em aberto,0,0,1,0,0,0,0,0,0
968,5948,64,80,Bahia,Salvador,41830-120,2023-11-07,NaN,0,0,...,Em aberto,0,0,1,0,0,0,0,0,0
969,5949,64,80,São Paulo,São Paulo,05014-001,2023-11-07,NaN,239,0,...,Em aberto,0,0,1,0,0,0,0,0,0
970,5954,63,82,São Paulo,São Paulo,04513-100,2023-11-07,NaN,239,0,...,Em aberto,0,0,1,0,0,0,0,0,0


## FUNIL_ASSINATURA_PIPEDRIVE_start_of_service

Substitui valores nulos pela data de início do contrato e transforma o tipo em datetime

In [43]:
data['FUNIL_ASSINATURA_PIPEDRIVE_start_of_service']

0      2023-01-27
1      2022-06-21
2      2021-07-05
3      2021-06-28
4      2021-06-28
          ...    
967    2023-11-06
968    2023-11-07
969    2023-11-07
970    2023-11-07
971    2023-11-08
Name: FUNIL_ASSINATURA_PIPEDRIVE_start_of_service, Length: 972, dtype: object

In [44]:
data['FUNIL_ASSINATURA_PIPEDRIVE_start_of_service'] = pd.to_datetime(data['FUNIL_ASSINATURA_PIPEDRIVE_start_of_service'])


for indice, valor in data['FUNIL_ASSINATURA_PIPEDRIVE_start_of_service'].items():
    if pd.isnull(valor):
        data.loc[indice, 'FUNIL_ASSINATURA_PIPEDRIVE_start_of_service'] = data.loc[indice, 'PESSOA_PIPEDRIVE_contract_start_date']
        
data['FUNIL_ASSINATURA_PIPEDRIVE_start_of_service']




0     2023-01-27
1     2022-06-21
2     2021-07-05
3     2021-06-28
4     2021-06-28
         ...    
967   2023-11-06
968   2023-11-07
969   2023-11-07
970   2023-11-07
971   2023-11-08
Name: FUNIL_ASSINATURA_PIPEDRIVE_start_of_service, Length: 972, dtype: datetime64[ns]

## FUNIL_ASSINATURA_PIPEDRIVE_lost_time

Utilizada para fazer uma nova coluna em conjunto com outras

In [45]:
data['FUNIL_ASSINATURA_PIPEDRIVE_lost_time']

0      2023-08-26
1      2023-09-20
2      2021-09-30
3      2023-10-16
4      2022-02-07
          ...    
967    2023-12-06
968    2023-12-06
969    2023-12-06
970    2023-12-06
971    2023-12-06
Name: FUNIL_ASSINATURA_PIPEDRIVE_lost_time, Length: 972, dtype: object

## FUNIL_ONBOARDING_PIPEDRIVE_add_time

Tudo certo por aqui, nada a acrescentar

In [48]:
data['FUNIL_ONBOARDING_PIPEDRIVE_add_time']

0      2021-09-16 18:15:11
1      2022-06-21 23:48:22
2      2021-09-16 18:28:04
3      2021-09-16 18:04:29
4      2021-09-16 18:05:03
              ...         
967    2023-11-06 18:34:05
968    2023-11-07 12:22:13
969    2023-11-07 13:05:44
970    2023-11-07 20:34:02
971    2023-11-08 19:03:40
Name: FUNIL_ONBOARDING_PIPEDRIVE_add_time, Length: 972, dtype: object

## FUNIL_ONBOARDING_PIPEDRIVE_activities_count

Tudo certo por aqui, nada a acrescentar

In [53]:
data['FUNIL_ONBOARDING_PIPEDRIVE_activities_count']

0      0
1      0
2      0
3      2
4      0
      ..
967    0
968    0
969    0
970    0
971    0
Name: FUNIL_ONBOARDING_PIPEDRIVE_activities_count, Length: 972, dtype: int64

## ATENDIMENTOS_AGENDA_Qde Todos Atendimentos

Tudo certo por aqui, nada a acrescentar

In [54]:
data['ATENDIMENTOS_AGENDA_Qde Todos Atendimentos']

0       2
1      25
2       0
3       0
4       0
       ..
967     0
968     1
969     0
970     2
971     0
Name: ATENDIMENTOS_AGENDA_Qde Todos Atendimentos, Length: 972, dtype: int64

## ATENDIMENTOS_AGENDA_Qde Atendimento Médico

Tudo certo por aqui, nada a acrescentar


In [56]:
data['ATENDIMENTOS_AGENDA_Qde Atendimento Médico']

0      0
1      1
2      0
3      0
4      0
      ..
967    0
968    0
969    0
970    0
971    0
Name: ATENDIMENTOS_AGENDA_Qde Atendimento Médico, Length: 972, dtype: int64

## ATENDIMENTOS_AGENDA_Faltas Atendimento Médico

Tudo certo por aqui, nada a acrescentar

In [57]:
data['ATENDIMENTOS_AGENDA_Faltas Atendimento Médico']


0      0
1      0
2      0
3      0
4      0
      ..
967    0
968    0
969    0
970    0
971    0
Name: ATENDIMENTOS_AGENDA_Faltas Atendimento Médico, Length: 972, dtype: int64

## ATENDIMENTOS_AGENDA_Datas Atendimento Médico

Tudo certo por aqui, nada a acrescentar

In [58]:
data['ATENDIMENTOS_AGENDA_Datas Atendimento Médico']

0            Nunca ocorreu
1      2023-01-12 17:00:00
2            Nunca ocorreu
3            Nunca ocorreu
4            Nunca ocorreu
              ...         
967          Nunca ocorreu
968          Nunca ocorreu
969          Nunca ocorreu
970          Nunca ocorreu
971          Nunca ocorreu
Name: ATENDIMENTOS_AGENDA_Datas Atendimento Médico, Length: 972, dtype: object